In [2]:
import requests
from bs4 import BeautifulSoup
from retry import retry

import pandas as pd
import numpy as np
import copy
import time

In [3]:
# 音を出す
def sound():
    !rundll32 user32.dll,MessageBeep

In [ ]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 10
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

1ページ目：6.873667478561401秒
総取得件数：140
完了：10.0%
残り時間：0時間1分1秒

2ページ目：6.996063232421875秒
総取得件数：491
完了：20.0%
残り時間：0時間0分55秒

3ページ目：7.071232080459595秒
総取得件数：723
完了：30.0%
残り時間：0時間0分48秒

4ページ目：6.808379888534546秒
総取得件数：963
完了：40.0%
残り時間：0時間0分41秒

5ページ目：9.646261215209961秒
総取得件数：1303
完了：50.0%
残り時間：0時間0分37秒

6ページ目：6.794625759124756秒
総取得件数：1528
完了：60.0%
残り時間：0時間0分29秒

7ページ目：7.934275150299072秒
総取得件数：1888
完了：70.0%
残り時間：0時間0分22秒

8ページ目：5.4975621700286865秒
総取得件数：2038
完了：80.0%
残り時間：0時間0分14秒



In [ ]:
columns = ['建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積']
suumo_tokyo = pd.DataFrame(data_samples, columns=columns)

In [ ]:
suumo_tokyo

In [ ]:
suumo_tokyo.to_csv(f'suumo{max_page}.csv', columns=columns, index=False)

In [4]:
suumo_tokyo = pd.read_csv('suumo2903.csv')

In [5]:
suumo_tokyo

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積
0,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2
1,ドリームハウス熊野前,東京都荒川区東尾久５,日暮里・舎人ライナー/熊野前駅 歩3分,ＪＲ高崎線/尾久駅 歩18分,ＪＲ山手線/田端駅 歩20分,築6年,3階建,2階,8.9万円,3000円,-,-,1LDK,33.94m2
2,東京メトロ千代田線 町屋駅 9階建 築17年,東京都荒川区町屋２,東京メトロ千代田線/町屋駅 歩3分,京成本線/町屋駅 歩2分,都電荒川線/荒川二丁目駅 歩3分,築17年,9階建,8階,11.3万円,5000円,11.3万円,11.3万円,1LDK,41.04m2
3,いずみマンションII,東京都荒川区町屋２,東京メトロ千代田線/町屋駅 歩6分,京成本線/新三河島駅 歩14分,ＪＲ山手線/西日暮里駅 歩23分,築17年,9階建,8階,11.3万円,5000円,11.3万円,11.3万円,1LDK,41.04m2
4,Sepia Palace,東京都荒川区東日暮里６,ＪＲ山手線/日暮里駅 歩8分,ＪＲ常磐線/三河島駅 歩8分,ＪＲ山手線/西日暮里駅 歩13分,新築,3階建,1階,11.9万円,5000円,-,11.9万円,1LDK,33.53m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222183,光洋荘,東京都文京区本郷５,東京メトロ丸ノ内線/本郷三丁目駅 歩6分,都営大江戸線/本郷三丁目駅 歩6分,NaN,築37年,1階建,1階,7万円,-,7万円,7万円,1DK,25m2
222184,第一旭荘,東京都目黒区柿の木坂１,東急東横線/都立大学駅 歩7分,東急東横線/学芸大学駅 歩15分,多摩01/めぐろ区民キャンパス 歩4分,築51年,2階建,2階,8万円,2000円,8万円,8万円,ワンルーム,24.79m2
222185,第一旭荘,東京都目黒区柿の木坂１,東急東横線/都立大学駅 歩7分,東急東横線/学芸大学駅 歩15分,多摩01/めぐろ区民キャンパス 歩4分,築51年,2階建,1階,9.5万円,2000円,9.5万円,9.5万円,ワンルーム,28.09m2
222186,Ｊ＆Ａ南葛西,東京都江戸川区南葛西３,東京メトロ東西線/葛西駅 歩24分,ＪＲ京葉線/葛西臨海公園駅 歩23分,東京メトロ東西線/西葛西駅 歩28分,築30年,4階建,4階,9.5万円,4000円,9.5万円,-,3DK,65.66m2


In [79]:
# 管理費の'-'を0に置き換える
# 管理費をint型に変換する
suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'] = suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'].replace('-','0')
suumo_tokyo['管理費'] = suumo_tokyo['管理費'].str.replace('円','').astype(int)

In [86]:
# 新築は築年数0年にする
# 築年数をint型に変換する
suumo_tokyo.loc[suumo_tokyo['築年数']=="新築",'築年数'] = suumo_tokyo.loc[suumo_tokyo['築年数']=="新築",'築年数'].replace('新築','0')
suumo_tokyo['築年数'] = suumo_tokyo['築年数'].str.replace('築','').str.replace('年','').astype(int)

In [51]:
# 地下変数を作った
suumo_tokyo['地下'] = np.zeros(suumo_tokyo.shape[0])
suumo_tokyo.loc[suumo_tokyo['階数'].map(lambda x: x.find('地下'))==0,'地下'] = suumo_tokyo.loc[suumo_tokyo['階数'].map(lambda x: x.find('地下'))==0,'階数'].map(lambda x: x[x.find('地下')+2:x.find('地上')]).astype(float)

In [ ]:
# 地上変数を作る
suumo_tokyo['地上'] = np.zeros(suumo_tokyo.shape[0])


In [58]:
# 地上変数を作成中
suumo_tokyo['階数'].map(lambda x: x[x.find('階建')-1])

0       2
1       3
2       9
3       9
4       3
       ..
2575    2
2576    3
2577    3
2578    3
2579    3
Name: 階数, Length: 2580, dtype: object